In [ ]:
import orjson, os
with open("local.settings.json") as f:
    os.environ.update(orjson.loads(f.read())["Values"])

# Database

In [ ]:
from sqlalchemy import create_engine
engine = create_engine(os.environ['DATABIND_SQL_KEYSTONE'])
schema = 'keystone'
table_name = 'Audience'
columns = []

In [ ]:
from sqlalchemy import MetaData, Table, select
import pandas as pd

# Create a MetaData object
metadata = MetaData()

# Reflect the table
table = Table(table_name, metadata, schema=schema, autoload_with=engine)

# select
columns = ["id"]
selected_columns = [table.c[col] for col in columns]
stmt = select(*selected_columns)

# Execute the query
with engine.connect() as conn:
    result = conn.execute(stmt)
    # Use mappings() to fetch rows as dictionaries
    rows = [row for row in result.mappings()]

audiences = pd.DataFrame(rows)
audiences = audiences.to_numpy().flatten()

# Table

In [ ]:
import os
from azure.data.tables import TableClient

def get_partition_keys_from_connection_string(conn_str: str, name_filter: str = "orchestrator_esquire_audience") -> list[str]:
    """
    Returns list of dicts with PartitionKey and name where name == name_filter.

    :param sas_url: Full SAS URL to the Azure Table
    :param name_filter: The value to match in the 'name' field
    :return: List of dicts: [{'PartitionKey': ..., 'name': ...}, ...]
    """
    table_client = TableClient.from_connection_string(conn_str, table_name="productionInstances")
    select_fields = ["PartitionKey", "Name"]

    results = []

    for entity in table_client.list_entities(select=select_fields):
        pk = entity.get("PartitionKey")
        name = entity.get("Name")
        if pk and name:
            results.append({"PartitionKey": pk, "name": name})

    return pd.DataFrame(results)



In [ ]:
prod_info = get_partition_keys_from_connection_string(os.getenv("AzureWebJobsStorage", ""))
prod_info = prod_info[prod_info['name'] == 'orchestrator_esquire_audience']
prod_keys = prod_info['PartitionKey'].values
display(len(prod_keys))

# Compare

In [ ]:
absent_audiences = set([key.split(':')[0] for key in prod_keys]) - set(audiences)
display(len(set([key.split(':')[0] for key in prod_keys]) & set(audiences)))
display(len(absent_audiences))
remove_keys = set([key for key in prod_keys if key.split(':')[0] in absent_audiences])
display(f"{len(remove_keys) / len(prod_keys):.2%} set to be removed")

# Delete

In [ ]:
import requests

for id in remove_keys:
    print(id)
    resp = requests.delete(
        url="https://esquire-auto-audience.azurewebsites.net/runtime/webhooks/durabletask/instances/{id}".format(
            id=id
        ),
        params={
            "taskHub": "production",
            "connection": "Storage",
            "code": os.environ['AZFUNC_MASTER_CODE'],
        },
    )